In [17]:
import fiftyone as fo
import fiftyone.zoo as foz
import os
import logging

logging.basicConfig(level=logging.DEBUG)

COCO_CLASSES = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']

In [11]:
safety_trigger = True

In [12]:
PROJECT_NAME = 'person'
ROOT_DIR = '/home/mosminin/datasets/COCO2017'



coco_dataset_dir = os.path.join(ROOT_DIR, 'coco2017_all_classes')
dataset_name = f'coco_{PROJECT_NAME}'

# Настройки экспорта
TARGET_CLASSES = ['person'] # Список нужных классов
yolov4_export_dir = os.path.join(ROOT_DIR, f'coco2017_yolov4_{PROJECT_NAME}') # Целевая папка
label_field = "ground_truth"

# ПАРСИНГ ДАТАСЕТА COCO-2017 ДЛЯ YOLOV4

## СКАЧИВАЕМ ПОЛНЫЙ ДАТАСЕТ COCO-2017

In [6]:
if not safety_trigger:
    # Выполнение кода грохнет текущий датасет и начнёт долгое скачивание заново!
    dataset = foz.load_zoo_dataset(
        "coco-2017",
        splits=["train", "validation", "test"],
        dataset_dir=coco_dataset_dir,
        dataset_name=dataset_name,
        dataset_type=fo.types.COCODetectionDataset,
        classes=COCO_CLASSES,
        drop_existing_dataset=True,
        # only_matching=False,
        shuffle=True,
        max_samples=1000
    )
    safety_trigger = True
else:
    print('Check safety trigger!')

Check safety trigger!


## ПООЧЕРЁДНО ЗАГРУЖАЕМ КАЖДЫЙ СПЛИТ И ЭКСПОРТИРУЕМ В СООТВЕТСТВУЮЩУЮ ПАПКУ

In [22]:
dataset = fo.load_dataset(dataset_name)

In [24]:
import shutil
from tqdm.notebook import tqdm
from pathlib import Path
from time import sleep

splits = ['train', 'validation', 'test'] # Splits for export
paths = {}

for split in splits:
    split_view = dataset.match_tags(split)
    if split == 'validation':
        split = 'valid'  # For Yolo compatibility
    export_dir = Path(os.path.join(yolov4_export_dir, split))
    logging.info(f'{export_dir=}')
    split_view.export(
        export_dir=str(export_dir),
        dataset_type=fo.types.YOLOv4Dataset,
        label_field=label_field,
        classes=TARGET_CLASSES,
    )

    yolov4_export_dir = Path(yolov4_export_dir)
    split_path = Path(os.path.join(yolov4_export_dir, split))
    data_path = Path(os.path.join(yolov4_export_dir, split,'data'))  # Abs path
    logging.debug(f'{split_path=}')
    logging.debug(f'{data_path=}')

    for file_path in split_path.glob('*'):
        if file_path.is_dir():
            continue
        file_path.unlink()
        logging.info(f'{file_path} is deleted')

    # Читаем все .jpg файлы в /data
    logging.info(f'Reading all files in data_path to file_list')
    file_list = []
    count = 0
    for count, file_path in enumerate(data_path.glob('*.jpg')):
        file_list.append(str(file_path.resolve()))
        count += 1
    logging.info(f'{count} file(s) added in file_list!')

    # Записываем список файлов в {split}.txt
    logging.info(f'Writing file_list to annotations file')
    annotations_file = yolov4_export_dir / f'{split}.txt'
    with open(annotations_file, 'w') as f:
        count = 0
        for file_path in file_list:
            # print(f'{file_path=}')
            f.write(file_path+'\n')
            count += 1
    logging.info(f'{count} file(s) added in file_list!')

    # Добавляем пути к сплитам в словарь
    paths[split] = str(annotations_file)




num_classes = len(TARGET_CLASSES)
classes_file_name = yolov4_export_dir / 'obj.names'
data_file_name = yolov4_export_dir / 'obj.data'
backup_dir_name = yolov4_export_dir / 'backup'

# Создать папку backup
backup_dir_name.mkdir(parents=True, exist_ok=True)
paths['backup'] = str(backup_dir_name)

# Сформировать obj.names и obj.data

with open(classes_file_name, "w") as f:
    for name in TARGET_CLASSES:
        f.write(name)

with open(data_file_name, "w") as f:
    # Добавить в obj.data кол-во классов, пути на аннотации и бэкап
    f.write(f'classes = {num_classes}\n')
    for key, value in paths.items():
        f.write(f'{key} = {value}\n')
    f.write('names = obj.names\n')




INFO:root:export_dir=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train')


   8% |█/-------------|   81/1000 [102.6ms elapsed, 1.2s remaining, 789.6 samples/s] 

/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'teddy bear' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'banana' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'clock' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'refrigerator' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'chair' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs

  29% |████|----------|  294/1000 [404.1ms elapsed, 970.4ms remaining, 727.5 samples/s] 

/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'frisbee' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'bird' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'motorcycle' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'keyboard' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'snowboard' not in provided classes
  warnings.warn(msg)


...

INFO:eta.core.utils: 100% |███████████████| 1000/1000 [1.3s elapsed, 0s remaining, 782.9 samples/s]         
DEBUG:root:split_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train')
DEBUG:root:data_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train/data')
INFO:root:/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train/obj.names is deleted
INFO:root:/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train/images.txt is deleted
INFO:root:Reading all files in data_path to file_list
INFO:root:1000 file(s) added in file_list!
INFO:root:Writing file_list to annotations file
INFO:root:1000 file(s) added in file_list!
INFO:root:export_dir=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/valid')


 100% |███████████████| 1000/1000 [1.3s elapsed, 0s remaining, 753.1 samples/s]         


INFO:eta.core.utils: 100% |███████████████| 1000/1000 [1.3s elapsed, 0s remaining, 753.1 samples/s]         
DEBUG:root:split_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/valid')
DEBUG:root:data_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/valid/data')
INFO:root:/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/valid/obj.names is deleted
INFO:root:/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/valid/images.txt is deleted
INFO:root:Reading all files in data_path to file_list
INFO:root:1000 file(s) added in file_list!
INFO:root:Writing file_list to annotations file
INFO:root:1000 file(s) added in file_list!
INFO:root:export_dir=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/test')


 100% |███████████████| 1000/1000 [229.5ms elapsed, 0s remaining, 4.4K samples/s]      


INFO:eta.core.utils: 100% |███████████████| 1000/1000 [229.5ms elapsed, 0s remaining, 4.4K samples/s]      
DEBUG:root:split_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/test')
DEBUG:root:data_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/test/data')
INFO:root:/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/test/obj.names is deleted
INFO:root:/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/test/images.txt is deleted
INFO:root:Reading all files in data_path to file_list
INFO:root:1000 file(s) added in file_list!
INFO:root:Writing file_list to annotations file
INFO:root:1000 file(s) added in file_list!


## СКРИПТ ДЛЯ РАСПРЕДЕЛЕНИЯ ДАТАСЕТА

- dataset   - valid - .jpg / .txt
            - train - .jpg / .txt
            - test  - .jpg

    - Для каждой папки перебрать .jpg и составить список train/valid/test.txt
    - Файлы со списками сохранить в dataset
    - В dataset папку backup
    - В dataset создать файл obj.data:
        classes =   {кол-во_классов}
        train =     {абсолютный пусть до train.txt}
        valid =     ...
        names =     {абсолютный пусть до obj.names}
        backup =    {абсолютный пусть до папки backup}

# БЭКАПИЛКА

In [ ]:
import os
import time
import tarfile

def make_archive(dir_path):
    now = time.strftime('%Y_%m_%d_%H%M%S')
    archive_name = f'{dir_path}_{now}.tar'
    i = 0
    while os.path.exists(archive_name):
        i += 1
        archive_name = f'{dir_path}_{now}_{i}.tar'
    with tarfile.open(archive_name, 'w') as archive:
        archive.add(dir_path, arcname=os.path.basename(dir_path), mode='a')